# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    Input=open(infile)
    H = nx.DiGraph()
    edges = {}
    ###read graph
    for line in Input:
        if(line.startswith('n')):
            initi,node,demand = line.split(" ")
            H.add_node(node)
            H.node[node]['demand']=int(demand)
        if(line.startswith('a')):
            initi,tail,head,lbound,capacity,cost = line.split(" ")
            if(tail not in H.nodes()):
                H.add_node(tail)
                H.node[tail]['demand']=0 
            if(head not in H.nodes()):
                H.add_node(head)
                H.node[head]['demand']=0 
            if(not H.has_edge(tail,head)):
                H.add_edge(tail,head)
            if tail in edges.keys():
                if head in edges[tail]:
                    edges[tail][head].append((int(capacity),int(cost),int(lbound)))
                else:
                    edges[tail][head]=[]
                    edges[tail][head].append((int(capacity),int(cost),int(lbound)))
            else:
                edges[tail]={}
                edges[tail][head]=[]
                edges[tail][head].append((int(capacity),int(cost),int(lbound)))
    
    ##add extra nodes
    for i in list(H.edges_iter()):
        tail,head= i
        #print("head:",head,"tail",tail,edges[head][tail])
        numEdges = len(edges[tail][head])
        if(numEdges>1):
            H.remove_edge(tail,head)
            for k in range(0,numEdges):
                newnodename = tail+'-'+str(k)+'-'+head
                H.add_node(newnodename)
                H.node[newnodename]['demand']=0
                H.add_edge(tail,newnodename)
                H.add_edge(newnodename,head)
                capacity,cost,lbound=edges[tail][head][k]
                H.edge[tail][newnodename]['capacity'],H.edge[tail][newnodename]['weight'],H.edge[tail][newnodename]['lbound']=(capacity,cost,lbound)
                H.edge[newnodename][head]['capacity'],H.edge[newnodename][head]['weight'],H.edge[newnodename][head]['lbound']=(capacity,0,lbound)

        else:
            #H.add_edge(tail,head)
            H.edge[tail][head]['capacity'],H.edge[tail][head]['weight'],H.edge[tail][head]['lbound']=edges[tail][head][0]

    return H

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    prob = pulp.LpProblem("myProblem", pulp.LpMinimize)
    node_demand_map = {}
    expression_to_minimize = []
    edge_counter = 1 # make sure each variable has a unique name since some edges appear twice
    for i in G.nodes():
        node_demand_map[i] = {
                'into': [],
                'outof': [],
                'demand': G.node[i]['demand']
            }

    edge_counter = len(G.edges())
    for edge in G.edges():
        v,w=edge
        cap=G.edge[v][w]['capacity']
        cost=G.edge[v][w]['weight']
        lbound=G.edge[v][w]['lbound']
        flow = pulp.LpVariable("%s_%s_%s" % (v, w, edge_counter), lbound, cap)
        expression_to_minimize.append(cost*flow)
        node_demand_map[w]['into'].append(flow)
        node_demand_map[v]['outof'].append(flow)
        edge_counter -= 1

    for node, data in node_demand_map.iteritems():
        prob += (sum(data['into']) - sum(data['outof']) == data['demand'])

    # add expression to minimize
    prob += sum(expression_to_minimize)

    # solve and return min-cost
    status = prob.solve()
    #print LpStatus[status]
    return int(pulp.value(sum(expression_to_minimize)))

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
